<a href="https://colab.research.google.com/github/jyotidabass/Prakriti_classification/blob/main/Prakriti_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
#importing the libraries for matrix and dataframe handling, plotting, etc 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# importing the other miscellaneous libraries used 
import re
import warnings
warnings.filterwarnings("ignore")

# importing the NLTK related libraries and functions along with evaluation metircs
import nltk
import string
nltk.download("all")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [73]:
unbalanced_df = pd.read_csv('/content/sample_data/try_prakriti.csv')
unbalanced_df.head()

,Body build,Appetite,Sleep,Skin,Urine quality,stool quality,Prakriti Score,cleaned_description,Prakriti
0,Thin,varies between good and small,disturbed sleep,"dry, no lubrication",medium yellow,constipated,12V,"Thin,varies between good and small, disturbed ...",Vata
1,Thin,good and large,disturbed sleep,"dry, no lubrication",medium yellow,constipated,10V 2P,"Thin,good and large, disturbed sleep, dry no l...",Vata Pitta
2,Thin,small appetite or severely low,disturbed sleep,"dry, no lubrication",medium yellow,constipated,10V 2K,"Thin,small appetite or severely low, disturbed...",Vata Kapha
3,Thin,varies between good and small,moderately good sleep,"dry, no lubrication",medium yellow,constipated,9V 3P,"Thin,varies between good and small, moderately...",Pitta
4,Thin,varies between good and small,sound sleep,"dry, no lubrication",medium yellow,constipated,9V3K,"Thin,varies between good and small, sound slee...",Kapha


In [74]:
category_mapping = {  0	: "Vata",
                      1	: "Pitta",
                      2	: "Kapha",
                      3	: "Vata Pitta",	
                      4	: "Vata Kapha",	
                      5	: "Pitta Vata",	
                      6	: "Pitta Kapha",	
                      7	: "Kapha Vata",
                      8	: "Kapha Pitta",	
                      9	: "Vata Pitta Kapha"}

reverse_category_mapping = {"Vata":0,
                            "Pitta":1,
                            "Kapha":2,
                            "Vata Pitta":3,	
                            "Vata Kapha":4,	
                            "Pitta Vata":5,	
                            "Pitta Kapha":6,	
                            "Kapha Vata":7,
                            "Kapha Pitta":8,	
                            "Vata Pitta Kapha":9}

In [75]:
#dropping the noise in the dataset by considering only the above mentioned categories

print(unbalanced_df.shape)

(12, 9)


In [76]:
def logistic_regression(x,y):
  from sklearn.linear_model import LogisticRegression

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the training dataset to the model
  lr_model = LogisticRegression()
  lr_model.fit(x_train,y_train)
  lr_predict = lr_model.predict(x_test)
  lr_pred_prob = lr_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, lr_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,lr_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,lr_predict))

  return y_test, lr_predict, lr_pred_prob

In [77]:
def logistic_regression_multiclass(x,y):
  from sklearn.linear_model import LogisticRegression

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  reg = 0.1

  #fitting the training dataset to the multiclass classification Logistic Regression model
  lr_model = LogisticRegression(C=1/reg, solver='lbfgs', multi_class='auto', max_iter=1000).fit(x_train,y_train)
  lr_predict = lr_model.predict(x_test)
  lr_pred_prob = lr_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, lr_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,lr_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,lr_predict))

  return y_test, lr_predict, lr_pred_prob

In [78]:
def naive_bayes(x,y):
  from sklearn.naive_bayes import MultinomialNB

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the tarining dataset to the model
  nb_model = MultinomialNB()
  nb_model.fit(x_train,y_train)
  nb_predict = nb_model.predict(x_test)
  nb_pred_prob = nb_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, nb_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,nb_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,nb_predict))

  return y_test, nb_predict, nb_pred_prob

In [79]:
def linear_svm(x,y):
  from sklearn.svm import LinearSVC

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the training dataset to the model
  svc_model = LinearSVC(random_state=42, max_iter=2000)
  svc_model.fit(x_train,y_train)
  svc_predict = svc_model.predict(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, svc_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,svc_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,svc_predict))

  return y_test, svc_predict

In [80]:
def decision_trees(x,y):
  from sklearn.tree import DecisionTreeClassifier

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the training dataset to the model
  dtree_model = DecisionTreeClassifier(max_depth = 30)
  dtree_model.fit(x_train,y_train)
  dtree_predict = dtree_model.predict(x_test)
  dtree_pred_prob = dtree_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, dtree_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test,dtree_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test,dtree_predict))

  return y_test, dtree_predict, dtree_pred_prob

In [81]:
def random_forest(x,y):
  from sklearn.ensemble import RandomForestClassifier

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the training dataset to the model
  rfc_model = RandomForestClassifier(random_state=42, max_depth = 30)
  rfc_model.fit(x_train,y_train)
  rfc_predict = rfc_model.predict(x_test)
  rfc_pred_prob = rfc_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, rfc_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test, rfc_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test, rfc_predict))

  return y_test, rfc_predict, rfc_pred_prob

In [82]:
def k_nearest_neighbours(x,y):
  from sklearn.neighbors import KNeighborsClassifier

  #splitting the dataset into training and test parts
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  #bag of words implmentation
  cv = CountVectorizer()
  x_train = cv.fit_transform(x_train).toarray()

  #TF-IDF implementation
  vector = TfidfTransformer()
  x_train = vector.fit_transform(x_train).toarray()
  x_test = cv.transform(x_test)

  #fitting the training dataset to the model
  knn_model = KNeighborsClassifier(algorithm='brute')
  knn_model.fit(x_train,y_train)
  knn_predict = knn_model.predict(x_test)
  knn_pred_prob = knn_model.predict_proba(x_test)

  #evaluation metrics for the dataset
  print("Validation Accuracy: ",accuracy_score(y_test, knn_predict))

  print("\n")
  print("*********** CONFUSION MATRIX **************")
  print(confusion_matrix(y_test, knn_predict))

  print("\n")
  print("*********** CLASSIFICATION REPORT **************")
  print(classification_report(y_test, knn_predict))

  return y_test, knn_predict, knn_pred_prob

In [83]:
def plot_roc_curve(y_test, y_pred, no_categories = 13, lw=2):

  #calculating the ROC curve and area for each class
  false_positive_rate = dict()
  true_positive_rate = dict()

  for i in range(no_categories):
    false_positive_rate[i], true_positive_rate[i], _ = roc_curve(y_test[:,i], y_pred[:, i])

  #Compute micro-average ROC curve and area under the curve
  false_positive_rate["micro"], true_positive_rate["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())

  #plotting the ROC Curves for each of the 13 main categories in our model
  for category in range(no_categories):
    plt.figure()
    plt.plot(false_positive_rate[category], 
             true_positive_rate[category], 
             color='deeppink',
             lw=lw)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Primary Category: ' + category_mapping[category])
  plt.show()

In [84]:
# the cleaned product description corresponds to the x value 
x = unbalanced_df['cleaned_description']
# the 13 labels/ product categories mentioned above correspond to the y value
y = unbalanced_df['Prakriti']

In [85]:
print("********** LOGISTIC REGRESSION **********")
y_test, lr_predict, lr_pred_prob = logistic_regression(x,y)

********** LOGISTIC REGRESSION **********
Validation Accuracy:  0.3333333333333333


*********** CONFUSION MATRIX **************
[[0 0 0]
 [1 1 0]
 [0 1 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Pitta       0.00      0.00      0.00         0
            Vata       0.50      0.50      0.50         2
Vata Pitta Kapha       0.00      0.00      0.00         1

        accuracy                           0.33         3
       macro avg       0.17      0.17      0.17         3
    weighted avg       0.33      0.33      0.33         3



In [86]:
print("********** MULTICLASS LOGISTIC REGRESSION **********")
y_test, lr_predict, lr_pred_prob = logistic_regression_multiclass(x,y)

********** MULTICLASS LOGISTIC REGRESSION **********
Validation Accuracy:  0.0


*********** CONFUSION MATRIX **************
[[0 0 0 0]
 [1 0 1 0]
 [0 0 0 0]
 [0 1 0 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Pitta       0.00      0.00      0.00       0.0
            Vata       0.00      0.00      0.00       2.0
      Vata Pitta       0.00      0.00      0.00       0.0
Vata Pitta Kapha       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0



In [87]:
print("TESTING LABELS: {}".format(y_test.shape))
print("PREDICTED LABELS: {}".format(lr_predict.shape))
print("PROBABILITY OF THE PREDICTED LABELS: {}".format(lr_pred_prob.shape))

TESTING LABELS: (3,)
PREDICTED LABELS: (3,)
PROBABILITY OF THE PREDICTED LABELS: (3, 9)


In [88]:
#converting the Test Classes (y_test) from Pandas Series object to Numpy array
y_test = y_test.to_numpy()
length = y_test.shape

#conerting the string classes into the respective numbers based on their mapping as described previously
for i in range(length[0]):
  y_test[i] = reverse_category_mapping[y_test[i]]
  lr_predict[i] = reverse_category_mapping[lr_predict[i]]

print("The last 10 actual labels: {}".format(y_test[:10]))
print("The last 10 predicted labels: {}".format(lr_predict[:10]))

The last 10 actual labels: [9 0 0]
The last 10 predicted labels: [0 3 1]


In [89]:
#converting the actual test labels into a binary 2d numpy array according to their classes

n_classes = 13
temp_array = [[0 for i in range(n_classes)] for i in range(length[0])]

j=0
for i in y_test:
  temp_array[j][i] = 1
  j+=1

#converting the temporary array into a numpy array 
y_test = np.array(temp_array)
print(y_test)

[[0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [93]:
print("********* NAIVE BAYES CLASSIFIER *********")
y_test, nb_predict, nb_pred_prob = naive_bayes(x,y)

********* NAIVE BAYES CLASSIFIER *********
Validation Accuracy:  0.0


*********** CONFUSION MATRIX **************
[[0 0 0 0]
 [1 0 1 0]
 [0 0 0 0]
 [0 1 0 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Pitta       0.00      0.00      0.00       0.0
            Vata       0.00      0.00      0.00       2.0
      Vata Pitta       0.00      0.00      0.00       0.0
Vata Pitta Kapha       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0



In [94]:
print("********* LINEAR SVM *********")
y_test, svm_predict = linear_svm(x,y)

********* LINEAR SVM *********
Validation Accuracy:  0.3333333333333333


*********** CONFUSION MATRIX **************
[[0 0 0]
 [1 1 0]
 [0 1 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Pitta       0.00      0.00      0.00         0
            Vata       0.50      0.50      0.50         2
Vata Pitta Kapha       0.00      0.00      0.00         1

        accuracy                           0.33         3
       macro avg       0.17      0.17      0.17         3
    weighted avg       0.33      0.33      0.33         3



In [95]:

print("********** DECISION TREES CLASSIFIER *************")
y_test, dtree_predict, dtree_pred_prob = decision_trees(x,y)

********** DECISION TREES CLASSIFIER *************
Validation Accuracy:  0.3333333333333333


*********** CONFUSION MATRIX **************
[[0 0 0]
 [1 1 0]
 [0 1 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Kapha       0.00      0.00      0.00         0
            Vata       0.50      0.50      0.50         2
Vata Pitta Kapha       0.00      0.00      0.00         1

        accuracy                           0.33         3
       macro avg       0.17      0.17      0.17         3
    weighted avg       0.33      0.33      0.33         3



In [96]:
print("********** RANDOM FOREST CLASSIFIER *************")
y_test, rfc_predict, rfc_pred_prob = random_forest(x,y)

********** RANDOM FOREST CLASSIFIER *************
Validation Accuracy:  0.0


*********** CONFUSION MATRIX **************
[[0 2 0]
 [0 0 0]
 [0 1 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

            Vata       0.00      0.00      0.00       2.0
      Vata Pitta       0.00      0.00      0.00       0.0
Vata Pitta Kapha       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0



In [97]:
print("********** K NEAREST NEIGHBOURS *************")
y_test, knn_predict, knn_pred_prob = k_nearest_neighbours(x,y)

********** K NEAREST NEIGHBOURS *************
Validation Accuracy:  0.0


*********** CONFUSION MATRIX **************
[[0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [0 1 0 0]]


*********** CLASSIFICATION REPORT **************
                  precision    recall  f1-score   support

           Kapha       0.00      0.00      0.00       0.0
           Pitta       0.00      0.00      0.00       0.0
            Vata       0.00      0.00      0.00       2.0
Vata Pitta Kapha       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0

